## 임상시험 데이터 확인
### 목적
임상시험 데이터 및 포맷, 스펙(스키마)를 확인하고 임상시험 데이터의 최종적 형태인 sas7bdat / xport를 다룬다.

### 준비
임상시험 데이터: [cafe.naver.com/dmisimportant/104](cafe.naver.com/dmisimportant/104)

### Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import secrets

### Variables
CRF 포맷, LAB 데이터 포맷, 데이터셋 파일 경로 설정

In [2]:
ornament="-"*10
hangul="가나다라마바사아자차카파타하"
ext=(".sas7bdat",".xport")
path_crf="C:/code/CUBEDEMO2017/CUBEDEMO_2017_dc_or_fmt.xlsx"
path_lab="C:/code/CUBEDEMO2017/CUBEDEMO_2017_dc_or_lar.xlsx"
path_set="C:/code/CUBEDEMO2017/SASSET/"

### 포맷 파일 로드
CRF 포맷 파일에서 데이터 도메인 확인

In [3]:
form_crf=pd.read_excel(path_crf)
form_lab=pd.read_excel(path_lab)
print(ornament,"domain formats:",os.linesep,form_crf.DOMAIN.unique())

---------- domain formats: 
 ['AE' 'AY' 'CM' 'CT' 'CY' 'DM' 'DS' 'EF' 'EG' 'ES' 'IE' 'IP' 'LB' 'LC'
 'LY' 'MH' 'MY' 'PD' 'PG' 'RN' 'SU' 'SV' 'VS']


### 데이터셋 확인
데이터셋 경로의 sas7bdat / xport 파일을 os.path 오브젝트로 로드하고 불량 파일이 있는지 확인

In [4]:
sasobj=[obj for obj in os.scandir(path_set) if any(map(obj.path.lower().__contains__,ext)) and obj.is_file()]
sasbad=[obj for obj in sasobj if obj.stat().st_size<3]
if len(sasbad)>1:raise Exception("exotic file exists")

sas7bdat / xport os.path 오브젝트를 도메인 이름: 데이터프레임의 딕셔너리로 로드하고 어떤 도메인의 데이터셋이 있는지 확인
- 데이터셋 내 string이 bytes이므로 utf-8로 변환하고, 이 과정에서 누락되는 레코드가 있는지도 확인

In [5]:
def _decode(filepath):
    data=pd.read_sas(filepath)
    nas=data.notna().value_counts().sum()
    bytecol=data.select_dtypes("object").columns
    data[bytecol]=data[bytecol].apply(lambda q:q.str.decode("utf-8"))
    if nas==data.notna().value_counts().sum():
        return data
    else:
        print(ornament,"error:",filepath)
        return None

data={os.path.splitext(obj.name)[0].upper():_decode(obj.path) for obj in sasobj}
print(ornament,"domain:",os.linesep,data.keys(),os.linesep,len(data),"domains")

---------- domain: 
 dict_keys(['AE', 'AY', 'CM', 'CT', 'CY', 'DA', 'DM', 'DS', 'DY', 'EF', 'EG', 'EN', 'ES', 'IE', 'IP', 'LB', 'LC', 'LY', 'MH', 'MY', 'PD', 'PG', 'RN', 'SN', 'SU', 'SV', 'VS']) 
 27 domains


### 도메인별 데이터셋 형태 확인
SN은 subject 이름, visit site를 포함함을 알 수 있음

In [6]:
sn=data["SN"]
sn.sample(10)

,SUBJID,VISIT,SNNAME,SNDTC
20,S-MJ-002,5007.0,김민지,2017-06-20
5,S-2Z-016,5007.0,황태중,2017-02-21
24,S-US-002,5007.0,이상엽,2017-03-23
16,S-3Z-048,5007.0,송나경,2017-06-29
22,S-MJ-010,3006.0,김민지,2017-06-14
9,S-2Z-028,5007.0,공우석,2017-02-20
19,S-MJ-001,5007.0,김민지,2017-03-24
4,S-2Z-013,5007.0,신서란,2017-02-20
6,S-2Z-017,3006.0,황태중,2017-02-21
10,S-2Z-029,3006.0,공우석,2017-02-20


#### SN: SNNAME 가리기
Subject 이름을 가림

In [7]:
def _aname(name,n=2,chars=hangul):
    suffix="".join([secrets.choice(chars) for q in range(n)])
    return name[:n]+suffix

sn_snname_mapper={name:_aname(name) for name in sn.SNNAME.unique()}
sn.SNNAME.replace(sn_snname_mapper,inplace=True)
data["SN"].sample(10)

,SUBJID,VISIT,SNNAME,SNDTC
6,S-2Z-017,3006.0,황태가나,2017-02-21
19,S-MJ-001,5007.0,김민가마,2017-03-24
20,S-MJ-002,5007.0,김민가마,2017-06-20
25,S-US-003,5007.0,이상아사,2017-03-23
17,S-4Z-001,3006.0,원유타아,2017-03-07
7,S-2Z-017,5007.0,황태가나,2017-02-21
11,S-2Z-029,5007.0,공우마아,2017-02-20
16,S-3Z-048,5007.0,송나다가,2017-06-29
13,S-2Z-034,3006.0,송나다가,2017-02-20
24,S-US-002,5007.0,이상아사,2017-03-23


#### SN: SUBJID 추출
SN의 SUBJID를 각 도메인의 인덱스로 씀
- SN.SUBJID 개수를 subject 수로 가정함

In [8]:
ix=sn.SUBJID.unique()
print(ix)
print(ornament,"total subjects:",len(ix))

['S-1Z-031' 'S-1Z-034' 'S-2Z-006' 'S-2Z-012' 'S-2Z-013' 'S-2Z-016'
 'S-2Z-017' 'S-2Z-023' 'S-2Z-028' 'S-2Z-029' 'S-2Z-032' 'S-2Z-034'
 'S-2Z-035' 'S-3Z-013' 'S-3Z-048' 'S-4Z-001' 'S-MJ-001' 'S-MJ-002'
 'S-MJ-009' 'S-MJ-010' 'S-US-002' 'S-US-003' 'S-US-007']
---------- total subjects: 23


#### DM: 내용 확인
DM은 인구학적 정보를 포함하며, 왜인지 인덱스(SN.SUBJID)에 부합하지 않는 row가 많음
- 인구학적 정보는 시험 초기에 수집되므로 인덱스에 해당하지 않는 row가 있으면 안됨

In [9]:
dm=data["DM"]
dm
dm[dm.SUBJID.isin(ix)]

,SUBJID,BRTHDTC,AGE,SEX,FERTILE
23,S-1Z-031,1984-09,32.0,2.0,1.0
26,S-1Z-034,1977-11,38.0,2.0,1.0
37,S-2Z-006,1975-04,41.0,2.0,2.0
43,S-2Z-012,1977-10,39.0,2.0,1.0
44,S-2Z-013,1991-07,25.0,1.0,NaN
47,S-2Z-016,1987-02,29.0,1.0,NaN
48,S-2Z-017,1983-03,33.0,1.0,NaN
54,S-2Z-023,1998-04,17.0,1.0,NaN
58,S-2Z-028,1989-04,27.0,2.0,2.0
59,S-2Z-029,1975-04,41.0,1.0,NaN


#### AE: 내용 확인
AE는 시험간 subject 별 adverse effect 정보를 포함
- 컬럼(e.g. form)별 숫자 코딩된 categorical data가 있음
- 상기에 따라 MedDRA 코딩된 컬럼이 있음
- 숫자 코딩의 기준은 CRF Format에 정의되어 있음

In [10]:
ae=data["AE"]
print(ae.loc[:,[col for col in ae if col.startswith("AE")]].sample(10))
print(ae[ae.SUBJID.isin(ix)][["SUBJID","INV_PT"]].sample(10))
form_crf[form_crf.DOMAIN=="AE"].sample(10)

                         AETERM     AESTDTC  AETEAE  AEOUT     AEENDTC  AESER  \
116  Hartmann's pouch procedure  2017-02-02     2.0    2.0         NaN    4.0   
158                         NaN         NaN     NaN    4.0         NaN    4.0   
110                         NaN         NaN     NaN    NaN         NaN    NaN   
97                  Sore throat  2016-02-25     2.0    3.0  2016-02-27    1.0   
132              Bruise of head  2016-02-UK     2.0    3.0  2017-02-07    1.0   
144                      Nausea  2016-06-22     1.0    5.0         NaN    1.0   
141                      Stroke  2016-02-25     2.0    1.0         NaN    1.0   
52                     AETEST01  2016-12-20     2.0    4.0  2017-02-UK    1.0   
149                 Eye allergy  2017-02-02     2.0    5.0         NaN    7.0   
130                Head banging  2016-02-UK     2.0    2.0         NaN    4.0   

     AESEV  AEREL  AEACN  AEACNOTH  
116    3.0    3.0    2.0       3.0  
158    NaN    4.0    4.0       4.0

,DOMAIN,FMTNAME,VARNAME,START,END,LABEL
27,AE,AESER,AESER,7,7,Other medically important event
20,AE,AEREL,AEREL,4,4,Related
22,AE,AESER,AESER,2,2,Death
23,AE,AESER,AESER,3,3,Hospitalization
0,AE,AEACN,AEACN,1,1,Dose increased
4,AE,AEACN,AEACN,5,5,Drug withdrawn
30,AE,AESEV,AESEV,3,3,Severe
11,AE,AEOUT,AEOUT,1,1,Fatal
28,AE,AESEV,AESEV,1,1,Mild
7,AE,AEACNOTH,AEACNOTH,1,1,None


#### AE: AE Code Labeling
AE의 코딩된 categorical AEs이 CRF 상에서 무엇을 의미하는지 확인하고자 함
- DB Specification 또는 CRF Format에서 컬럼별 코드 및 라벨을 컬럼: 코드: 라벨의 딕셔너리로 추출
- 딕셔너리에 따라 컬럼 코드를 라벨링
- 일견 AESER rate가 상당히 높음

In [11]:
def _get_label(form,var="VARNAME",kvp=["END","LABEL"]):
    label={}
    for varname in form[var].unique():
        kv=form[form[var]==varname].loc[:,kvp].values
        label[varname]={k:v for k,v in kv}
    return label

ae_label=_get_label(form_crf)
print(ae_label)

{'AEACN': {1: 'Dose increased', 2: 'Dose not changed', 3: 'Dose reduced', 4: 'Drug interrupted', 5: 'Drug withdrawn', 6: 'Not applicable', 7: 'Unknown'}, 'AEACNOTH': {1: 'None', 2: 'Drug treatment', 3: 'Non-drug treatment', 4: 'Drug and non-drug treatment'}, 'AEOUT': {1: 'Fatal', 2: 'Not recovered/Not resolved', 3: 'Recovered/Resolved', 4: 'Recovered/Resolved with sequelae', 5: 'Recovering/Resolving', 6: 'Unknown'}, 'AEREL': {1: 'Not related', 2: 'Unlikely related', 3: 'Possibly related', 4: 'Related'}, 'AESER': {1: 'No', 2: 'Death', 3: 'Hospitalization', 4: 'Life threatening', 5: 'Congenital anomaly or birth defect', 6: 'Significant disability', 7: 'Other medically important event'}, 'AESEV': {1: 'Mild', 2: 'Moderate', 3: 'Severe'}, 'AETEAE': {1: 'Yes', 2: 'No'}, 'AEYN': {1: 'Yes', 2: 'No'}, 'CMINDC': {1: 'Medical history', 2: 'Adverse event', 3: 'Hypertension', 4: 'Prophylaxis', 5: 'Supplement', 6: 'Other'}, 'CMONGO': {1: 'Checked'}, 'CTYN': {1: 'Yes', 2: 'No'}, 'CMYN': {1: 'Yes', 2:

In [12]:
ae_col_ambi=[col for col in ae if col in form_crf[form_crf.DOMAIN=="AE"].VARNAME.values]
ae[ae_col_ambi].replace(ae_label)

,AETEAE,AEOUT,AESER,AESEV,AEREL,AEACN,AEACNOTH
0,Yes,Not recovered/Not resolved,Life threatening,Severe,Unlikely related,Dose reduced,Non-drug treatment
1,Yes,Unknown,Death,Mild,Related,Drug interrupted,None
2,NaN,Recovering/Resolving,NaN,NaN,NaN,NaN,NaN
3,Yes,Fatal,Life threatening,Moderate,Unlikely related,Dose not changed,Drug treatment
4,Yes,Recovered/Resolved with sequelae,Hospitalization,Moderate,Possibly related,Dose reduced,Drug treatment
...,...,...,...,...,...,...,...
186,No,Unknown,No,Mild,Not related,Dose not changed,Drug treatment
187,Yes,Not recovered/Not resolved,Hospitalization,Mild,Unlikely related,Drug withdrawn,Drug treatment
188,Yes,Fatal,No,Mild,Unlikely related,Unknown,None
189,No,Not recovered/Not resolved,Other medically important event,Mild,Possibly related,Drug withdrawn,Drug and non-drug treatment


#### MH: Specification
MH 데이터의 스키마 불러오기
- 위와는 다르게 DB Specification을 사용: DB Spec.에 모든 데이터셋의 스키마가 전부 있음

In [13]:
path_spec="C:/code/CUBEDEMO2017/spec.xlsx"
spec=pd.read_excel(path_spec).loc[:,:"VIEW_TYPE"]
spec.CODE.sample(5)

109                             NaN
88                              NaN
152    1:None|2:Low|3:Middle|4:High
8                               NaN
140                             NaN
Name: CODE, dtype: object

- DB Spec-sheet의 CODE 컬럼 내용에 NaN과 문자열이 혼재함
- 차후 편한 앤코딩 및 디코딩을 위해 코딩 규칙이 있는 경우 (notna) 딕셔너리로 바꿈

In [14]:
spec["CODE"]=spec.CODE.apply(lambda q:dict(item.split(":") for item in q.split("|")) if pd.notna(q) else q).dropna()
spec.CODE.sample(5)

256    {'1': 'Yes', '2': 'No'}
69                         NaN
131                        NaN
120    {'1': 'Yes', '2': 'No'}
272                        NaN
Name: CODE, dtype: object

- MH 데이터셋 불러오기 및 주요 컬럼 확인

In [15]:
mh=data["MH"]
mh[[col for col in mh if col.startswith("MH")]].sample(5)

,MHTERM,MHONGO,MHENDTC,MHCONTRT,MHCO
27,바꾼용어,1.0,NaN,1.0,test11
42,Head banging,2.0,2012-10-14,1.0,NaN
62,NaN,2.0,NaN,1.0,NaN
91,Headache,1.0,NaN,1.0,NaN
80,Hand crushing,2.0,2016-09-UK,2.0,NaN


- Spec-sheet에서 데이터 컬럼 스키마를 냄

In [30]:
def _label(series,spec):
    spec=spec[spec.ITEMID==series.name]
    crfname=spec.CRF_LABEL.iat[0]
    name=spec.ITEM_LABEL.iat[0]
    code=spec.CODE.iat[0]
    visit=spec.VISIT.iat[0]
    return crfname,name,code,visit

mh_MHONGO_spec=_label(mh.MHONGO,spec)
print(ornament,"\nCRF Name: ",mh_MHONGO_spec[0],"\nForm Label: ",mh_MHONGO_spec[1],"\nCode: ",mh_MHONGO_spec[2])

---------- 
CRF Name:  Medical History 
Form Label:  Ongoing 
Code:  {'1': 'Yes', '2': 'No'}


- MH의 MHONGO는 CRF의 Medical History 부문에 있고, ongoing 여부를 묻는 binary 질문임

#### LB: 내용 확인
LB는 랩 테스트 결과를 포함
- VISIT, SEQ 등 랩 테스트 회차에 따른 롱 포맷으로 되어 있으며, 인덱스를 정할 때 유의해야 함
- LB.LBTEST에 해당 랩 테스트 명목이 나와 있음
- LB.LBORRES에 랩 결과 수치가 나와 있음
- 일부 categorical data 포함 (e.g. Occult Blood)

In [ ]:
lb=data["LB"]
lb.sample(10)

,SUBJID,VISIT,SEQ,LBTEST,LBORRES,LBNOR,LBCLSIG
2069,S-3Z-013,4.0,5.0,Leukocytes,9.00,1.0,NaN
2452,S-3Z-035,1.0,13.0,"Glomerular Filtration Rate, Estimated [Automat...",5.86,NaN,NaN
2458,S-3Z-037,1.0,1.0,Erythrocytes,3.70,2.0,1.0
1652,S-2Z-029,7.0,12.0,Creatinine,0.50,2.0,1.0
517,S-1Z-029,2.0,10.0,Bilirubin,1.70,1.0,NaN
361,S-1Z-026,2.0,17.0,Albumin [U],NaN,1.0,NaN
936,S-2Z-005,5.0,18.0,Occult blood[U],NaN,1.0,NaN
453,S-1Z-028,5.0,19.0,Protein[U],NaN,1.0,NaN
372,S-1Z-028,1.0,9.0,Alanine Aminotransferase,46.00,1.0,NaN
307,S-1Z-024,3.0,17.0,Albumin [U],NaN,1.0,NaN


In [ ]:
lb_desc=lb.groupby(["LBTEST"])["LBORRES"].agg(["mean","std","min","max","sum"])
lb_desc

,mean,std,min,max,sum
LBTEST,,,,,
Alanine Aminotransferase,336.257126,1409.494132,0.300,9083.352,64225.111
Albumin,335.514705,1577.394172,1.000,9954.012,63747.794
Albumin [U],NaN,NaN,NaN,NaN,0.000
Aspartate Aminotransferase,325.924723,1415.747206,1.000,9930.498,62251.622
Bilirubin,215.007593,1165.893477,0.400,9890.035,40636.435
Creatinine,184.471609,1056.242446,0.377,9596.921,35418.549
Erythrocytes,245.251260,1130.807728,1.000,7803.056,47088.242
"Glomerular Filtration Rate, Estimated [Automatical",5003.627111,3658.624184,782.108,9718.770,45032.644
"Glomerular Filtration Rate, Estimated [Automatically calculated]",82.635866,40.426010,0.080,194.760,14791.820


#### LB: Mockup Data 만들기
Subject Index와 LB를 이용해서 목업 데이터를 만듦
- SN.SUBJID를 기준으로 피험자당 10번의 랩 테스트를 가정함
- 편의상 DM 데이터를 결합
- 연습용 데이터 특성상 생성된 데이터는 현실성이 없음

In [ ]:
def _gen_mockup_value(desc,count):
    print(ornament,"generating values")
    return {q:np.random.normal(desc.loc[q]["mean"],desc.loc[q]["std"],count) for q in desc.index}

def _gen_mockup(desc,ix,count=10):
    data=_gen_mockup_value(desc,count=len(ix)*count)
    ix=pd.MultiIndex.from_product([ix,[q for q in range(1,count+1,1)]],names=["SUBJID","VISIT"])
    return pd.DataFrame(data,ix).reset_index()

_gen_mockup(lb_desc.dropna(),ix).merge(dm,on="SUBJID")

---------- generating values


,SUBJID,VISIT,Alanine Aminotransferase,Albumin,Aspartate Aminotransferase,Bilirubin,Creatinine,Erythrocytes,"Glomerular Filtration Rate, Estimated [Automatical","Glomerular Filtration Rate, Estimated [Automatically calculated]",...,Hemoglobin A1C,Leukocytes,Platelets,Protein,Specific Gravity[U],pH[U],BRTHDTC,AGE,SEX,FERTILE
0,S-1Z-031,1,388.378472,535.922432,-771.701886,375.192192,1390.871666,-1211.128103,2542.513236,79.300198,...,-241.414497,120.094771,1420.827243,523.733193,-139.071540,-97.200665,1984-09,32.0,2.0,1.0
1,S-1Z-031,2,-1018.008418,417.963924,707.704626,-1413.822525,-471.305554,118.731936,-1588.156122,121.642968,...,435.401395,-256.038511,1530.194364,-900.033360,-396.347002,-1745.910225,1984-09,32.0,2.0,1.0
2,S-1Z-031,3,2288.918589,738.532231,-125.374939,1073.681190,515.489574,1478.654463,6026.838795,50.649775,...,169.321307,-115.114176,1442.954558,-406.546638,-266.968893,188.926466,1984-09,32.0,2.0,1.0
3,S-1Z-031,4,1315.164599,-2203.422560,-1737.902812,1036.717322,354.442235,-175.940836,1193.574787,102.723784,...,1095.041104,-721.808365,-1321.772883,-525.645791,-1153.344891,448.667502,1984-09,32.0,2.0,1.0
4,S-1Z-031,5,-580.189818,1884.689726,-600.808936,1055.833174,175.088331,639.045493,4265.610510,127.519603,...,253.498656,-521.759966,939.539734,78.614015,-527.703472,401.153251,1984-09,32.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,S-US-007,6,3739.211479,1864.127650,69.021001,247.090336,1219.708155,-15.473255,2886.577649,123.259632,...,-1447.513794,-838.234266,96.269636,53.541509,-1069.591291,1567.665320,1981-01,36.0,1.0,NaN
226,S-US-007,7,-700.205813,2095.105398,1271.116886,767.753600,1205.189434,-2534.140389,7324.495147,44.664748,...,1067.484664,-91.830004,-2415.073425,-2215.880723,-55.227110,-302.263841,1981-01,36.0,1.0,NaN
227,S-US-007,8,-112.193337,-2883.701294,1871.828737,565.514639,552.763140,125.133130,4734.015709,71.108923,...,-604.466869,-654.926934,-1759.420495,-342.449691,1522.772961,-1830.195400,1981-01,36.0,1.0,NaN
228,S-US-007,9,-697.164591,-987.672775,-1497.078459,857.713624,66.436427,66.865164,7002.946540,50.842760,...,1118.497275,149.595112,970.617938,-1018.898139,-101.608095,89.288639,1981-01,36.0,1.0,NaN


### 후기
- 하나의 임상시험을 구성하는 데이터셋은 많으며, 각 데이터 연동을 위한 정의와 연동이 일목요연히 되어 있음
    - 수치형, 범주형 값이 혼재함
        - 범주형 데이터인데 숫자로 코딩되고 수치형 값과 기술적으로 구분되지 않는 것을 유의해야 함
    - 일반적이지 않은 범주형 값 (0,1 No,Yes(False,True)가 아닌 1,2 Yes,No)
    - 데이터셋 복잡도의 큰 요인 중 하나는 프로토콜의 site visit number로 생각됨
- 분석에 이용되는 실질적 데이터뿐만 아니라 엔드유저에게 해당 데이터가 어떻게 보이고 수집됐는지를 포함
- 프로토콜 이해를 선행하지 않고서는 정확히 데이터셋을 다루기 어려움
    - Visit Number / Window, Endpoint, Inclusion / Exclusion Criteria
    - Longitudinal Study 여부 (endpoint가 time series인지)
- 도메인 데이터와 DB Specification 내 각 도메인 정의를 연동할 수 있는 클래스 이용 필요